# Sedaro Studies Example Notebook
Demostrates Sedaro Studies support via the sedaro python client using a Jupyter notebook.

> Task
Where users can adapt them to do their own studies. It should be obvious where the top level override configuration is done as well as the feedback loop (in the case of the Monte Carlo). The notebooks should demonstrate the ability to run many simulations in parallel.  In the monte carlo case, this will be via batches of N sims that are run and then the results pulled and then another N run depending on feedback loop.
- Public jupyter notebooks for:
    - Wildfire Trade Space Analysis: Battery Size vs. Mass
    - Wildfire Monte Carlo: 
        - Vary uncertainty on:
            - Moment of magnetorquers
            - Mass of reaction wheels
            -  Initial position, velocity, and attitude
            - Vary random seeds of sensors by changing simulation-wide seen
        - To understand uncertainty of average pointing error


# Introduction

This notebook demostrates how to use the new Studies support added to Sedaro. 
The core of this support is the new API python client object **SimStudy**
SimStudy will generate and run a series of SimJobs in parallel up to account capacity limits. The remaining simjobs will be placed in a queue and will be execute when resources are available. 

# Setup

Requires
- Sedaro Account          --> https://www.sedaro.com
- Sedaro API Token        --> https://www.sedaro.com/#/account
- Wildfire Demo Branch ID --> login --> select/click or create workspace --> select/click Project: [DEMO] WildFire --> select/click  Repositories: [DEMO] Wildfire Scenarios --> copy main branch ID via clipboard icon (insert screen shot)
- Python 3.10+ installed  --> https://www.python.org
- Jupyter notebook or lab --> https://jupyter.org


## Pip requirements

In [ ]:
# create/activate a python venv if desired

# https://github.com/sedaro/sedaro-python
# Required
!python -m pip install -e sedaro pytest mathplotlib pandas sweetviz

In [ ]:
# or replace python with python3 if needed 
!python -m pip install -e sedaro pytest mathplotlib pandas sweetviz

In [ ]:
# optional 
!pip3 install sweetviz

In [31]:
import sedaro
import yaml
import json
import matplotlib.pyplot as plt
import pandas as pd

# optional
import sweetviz as sv

## Sedaro python client setup

In [36]:
Sedaro_api_host  = "http://localhost:80" # "api.sedaro.com"
Sedaro_api_token = "PKNxBcgPj8Hl6vzhJhTgDp.mPjNiu-ZmWxwm6buhu3pjic8aNyQaYnGWpaMgE7051uvCUVJI4SvzBasB_BzvoPnDcB7CqWDq4umZl7w9q5aFw"
sedaroAPI = sedaro.SedaroApiClient(api_key=Sedaro_api_token, host=Sedaro_api_host)

## Load wildfire scenario branch data

In [54]:
scenario_branch_id = "PKNxCWQL8N3cR5CG7YS6X3"
wildfire_scenario_branch = sedaroAPI.scenario(scenario_branch_id)

In [ ]:
wildfire_scenario_branch.study(sedaroAPI, scenario_branch_id)

# Overview of the Study objects

## Study
Create via
```
resource = f'/simulations/branches/{scenario_branch_id}/control/study/'
result = sedaroAPI.request.post(resource,
       body={
            "iterations": 3
            })
```
| Class Method | Arguments  | Returns | Description |
| :-: | :-: | :-: | :- | 
| start | ( iterations: int ) | StudyHandle | Starts study corresponding to the respective Sedaro Scenario Branch id. |
| status | ( job_id: str = None ) | StudyHandle |  Refreshes the local study status. |
| terminate | ( job_id: str = None ) | StudyHandle | Terminate latest running simulation job corresponding to the respective Sedaro Scenario Branch id. |
|  |  |  | If a `job_id` is provided, that simulation job will be terminated rather than the latest. |
| |
| results | (job_id: str = None) | StudyResult | Query latest scenario study result. If a `job_id` is passed, query for corresponding sim results rather than latest. |
| results_poll | ( job_id: str = None, retry_interval: int = 2) | StudyResult | Query latest scenario study result and wait for sim to finish if it's running. | 
| | | | If a `job_id` is passed, query for corresponding study results rather than latest. See `results` method for details on using the `streams` kwarg. |
| |
| stats_results | () | StudyStatsResult | Retuens a 'StudyStatsResult' instance of Statistics based on the study result data |


## StudyHandle

| Class Method | Arguments  | Returns | Description |
| :-: | :-: | :-: | :- | 
| get | (key, default=None) | StudyHandle | Returns a Study handle with the given key |
| status | (err_if_empty: bool = True) | StudyHandle | Refreshes the local study status. |
| terminate | () | StudyHandle | Terminate the running study. |
| |
| results | () | StudyResults | Query study results. |
| results_poll | (retry_interval: int = 2) | StudyResults | Query study results but wait for sim to finish if it's running. See `results` method for details on using the `streams` kwarg. |
| |
| stats_results | () | StudyStatsResult | Returns a StudyStatResult object |

## StudyResults
By default, this class will lazily load simulation results as requested
and cache them in-memory. Different caching options can be enabled with
the .set_cache method.
        
| Class Method | Arguments  | Returns | Description |
| :-: | :-: | :-: | :- | 
| id | () | int | Returns the id of the StudyResult Object |
| branch | () | str | Returns the branch id of the StudyResult Object |
| scenario_hash | () | str | Returns a unique hash |
| status | () | str | xx |
| date_created | () | datetime | xx |
| date_modified | () | datetime | xx |
| job_ids | () | List[int] | Returns a list of SimJob id's created by the Study.start method |
| iterations | () | int | Returns the number of SimJobs created by the Study.start method |
| set_cache | (cache: bool = True, cache_dir: str = None) | None | Set caching options for this study result. |
| | | | cache: Boolean option to turn caching on or off. |
| | | | cache_dir: Path to a directory for on-disk caching. |
| result | (id_: str, streams: Optional[List[Tuple[str, ...]]] = None) | SimulationResult | Query results for a particular simulation. |
| clear_cache | () | None | Clears the cache files in the cache_dir |
| summarize | () | None | Summarize these results in the console.' |


## StudyStatsResults
By default, this class will lazily load simulation results as requested
and cache them in-memory. Different caching options can be enabled with
the .set_cache method.

| Class Method | Arguments  | Returns | Description |
| :-: | :-: | :-: | :- | 
| id | () | int | Returns the id of the StudyResult Object |
| branch | () | str | Returns the branch id of the StudyResult Object |
| scenario_hash | () | str | Returns a unique hash |
| status | () | str | xx |
| date_created | () | datetime | xx |
| date_modified | () | datetime | xx |
| job_ids | () | List[int] | Returns a list of SimJob id's created by the Study.start method |
| iterations | () | int | Returns the number of SimJobs created by the Study.start method |
| set_cache | (cache: bool = True, cache_dir: str = None) | None | Set caching options for this study result. |
| | | | cache: Boolean option to turn caching on or off. |
| | | | cache_dir: Path to a directory for on-disk caching. |
| result | (id_: str, streams: Optional[List[Tuple[str, ...]]] = None) | SimulationResult | Query results for a particular simulation. |
| clear_cache | () | None | Clears the cache files in the cache_dir |
| summarize | () | None | Summarize these results in the console.' |
| stats_results | (_ids=None ) | dict[simjob_id, pandas.dataframe] | Packages the sim results into a dictory of simjob_id to sim results as a pandas dataframe |
| xx | xx | xx | xx |
| xx | xx | xx | xx |
| xx | xx | xx | xx |

# Overview of the overrides feature

## What do they do
- Change anything and choose how it is varied
 - Adds Model snapshotting
 - Choice is Deterministic between Simulation runs

## Paths
an addressing means to access a parameter of an Agents block. Currently there are two version:
- "path":  Agent Name / blockname / parameter name / sub series
 - Example: “Wildfire/Crosslink/minTimeBetweenOccurrences/min”
 - Uses “/“ to mimic file locations
 - More human readable
- "agent_key": agent_id.data.blocks.block_id.parameter.subseries
 - Example: “NT0LWIfSJ1RIenKpUmJEV.data.blocks.NV0bIfUUj9e6l-PX4ql1V.loadDefParams.power”
 - Uses ‘.’ for the separator
 - Similar to how Sedaro results streams are named

## Variables
Used to ensure consistany when assigning random values to parameters For example if the area of a solar panel is randomly assigned, then  

### Built-in

### User defined

In [32]:
variables = [
        {
            "name": "gyroHotTemp",
            "path": "Wildfire/Gyro/hotTempRating/degC",      
        },
        {
            "name": "gyro_Sandwich",
            "equals": "tasty"
        }
    ]

In [33]:
overrides = [
        {
            "path": "Wildfire/Gyro/hotTempRating/degC", # 100.0
            "fn": "=",
            "arg": 90.0
        },
    ...
]

In [34]:
overrides

[{'path': 'Wildfire/Gyro/hotTempRating/degC', 'fn': '=', 'arg': 90.0},
 Ellipsis]

## Available overrides

### fn

Executes the function and places the result into the provided Parameter Path

##### basic math operators +, -, *, /
##### Python random module functions

| Basic Math Operators | Random Module |  |   |   |
| :-: | - | - | - | - |
| + | choices | sample | randrange | uniform |
| - | triangular |expovariate | gammavariate | gauss |
| * | normalvariate | lognormvariate | vonmisesvariate | paretovariate| 
| / | weibullvariate |


### fn_chain
Performs the provided list of fn overrides in order, each time storing the result in the given parameter path   

### sim_index
Used to perform tradespace studies. The StudyJob object will create SimJobs each with an unique index. The **sim_index** override will use this index to select from a list of **fn** overrides to use for that SimJob.

The **sim_index**  is modulated by the size of the provided override fn list
> fn used = fn_list[ int(sim_index)%len(fn_list) ]


### copy_value_to (*Working name*)

### clamp

### clampRedo(*soon*)

# Example Studies

## Wildfire Tradespace Analysis: Battery Size vs. Mass effects on Agent maneuverability
### Optional: Load shared workspace to skip setup and running the study

#### Tradespace table used
#### Overrides used


### Setup

#### Deternmine Parameter Paths

#### Create tradespace table 

| Agent Parameter | Sim Alpha Value | Sim Beta Value | Sim Charlie Value | Parameter Path | Agent_ID_PATH |
| :-: | :-: | :-: | :-: | :-: | :-: |
| Battery Size | 9999 | 8888 | 7777 | xxxx | yyyy |
| Mass | 6666 | 5555 | 4444 | aaaa | bbbb |

#### Create overrides from tradespace table

In [ ]:
tradespace_overrides_dict = {}

In [ ]:
tradespace_overrides_block = wildfire_branch.OverrideSet.create(**tradespace_overrides_dict)

record the generated override block ID

#### Create Study 

In [ ]:
study_id = 'PK5Z4Mrx8Phf7ZbgGqy4Rr'
study_control_resource = f'/simulations/branches/{scenario_id}/control/study/{study_id}'

### Run Study

In [ ]:
study_status = sedaroAPI.request.get(study_control_resource)

In [ ]:
study_status

In [ ]:
study_job_ids = study_status['jobs']

In [ ]:
[ sedaroAPI.request.get(f'/simulations/branches/{scenario_id}/control/{job_id}') for job_id in study_job_ids]

### Load Study results

In [ ]:
study_alpha_results = studyjob.result(study_job_ids[0])

### Analyze Study results

In [ ]:
study_alpha_results.summarize()

In [ ]:
study_alpha_Wildfire_agent_result = study_first_results.agent('Wildfire')

In [ ]:
study_alpha_Wildfire_agent_result.stats(module='cdh')

In [ ]:
study_alpha_Wildfire_agent_result.blockname("Magnetometer").stats()

In [ ]:
study_alpha_Wildfire_agent_result.blockname("Magnetometer").scatter_matrix()

In [ ]:
plain_first_results =  wildfire_branch.simulation.results_plain(streams=[(wildfire_agent_id,'CDH')])

In [ ]:
with open("first_CDH_results.json", 'w') as writer:
    writer.write(str(plain_first_results))

In [ ]:
first_results = wildfire_branch.simulation.results(study_job_ids[1],[(wildfire_agent_id,'CDH')])

In [ ]:
first_results.summarize()

In [ ]:
first_results.agent('Wildfire').stats('cdh', make_histogram_plots=False)

In [ ]:
(first_results.agent('Wildfire').blockname('Fire: Chichen Itza').summarize())

In [ ]:
(first_results.agent('Wildfire').blockname('Fire: Chichen Itza').stats())

In [ ]:
first_results.agent('Wildfire').blockname('LaserComm-4').range['km'].stats(output_html=False)

In [ ]:
second_results = wildfire_branch.simulation.results(study_job_ids[1],[(wildfire_agent_id,'CDH')])

In [ ]:
second_results.agent('Wildfire').blockname('LaserComm-4').targetElevation['deg'].plot()

In [ ]:
tradespace_studyjob = wildfire_branch.study.results('PK5Z4Mrx8Phf7ZbgGqy4Rr')

In [ ]:
studyjob.summarize()

## Wildfire Monte Carlo Analysis: Understand uncertainty of average pointing error
Parameters to vary:
- Moment of magnetorquers
- Mass of reaction wheels
- Initial position, velocity, and attitude
- Vary random seeds of sensors by changing simulation-wide seen
- 
### Optional: Load shared workspace to skip setup and running the study

#### Tradespace table used
#### Overrides used

### Setup

#### Table of Agent parameters to vary

#### Determine how to vary the parameters

#### Create Overrides

#### Create Study

### Run Study

### Load Study Results

### Analyze Results

In [ ]:
monte_carlo_overrides_block = wildfire_branch.OverrideSet.create(**monte_carlo_overrides_dict)

# Appendix A: Python client plot/statistics features

## Agent level

## Block level

## Parameter level

## Study level